In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
parent_dir=""                            #to be changed if run on different computers

In [2]:
import matplotlib.pyplot as plt
import librosa
import librosa.display
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import glob
import keras
import sys
import time
import csv
import datetime
import keras.backend as K
import keras
import time 
import csv 
import datetime
from scipy.io import wavfile
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from matplotlib.pyplot import specgram
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense,Embedding
from keras.layers import LSTM
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Input,Flatten,Dropout,Activation
from keras.layers import Conv1D,MaxPooling1D,AveragePooling1D,Conv2D,MaxPool1D,MaxPool2D,Conv2D
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import accuracy_score,confusion_matrix
from keras import regularizers
from collections import Counter
from tqdm import tqdm
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix
from librosa import display
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential,Model
from keras.layers import Dropout,Flatten,Dense,Input,Convolution2D,MaxPooling2D,Activation,concatenate,LSTM
from keras.layers.normalization import BatchNormalization
from tqdm import tqdm_notebook
from sklearn.svm import SVC

Using TensorFlow backend.


In [5]:
def extract_features(file_path):
    win_size=1024
    hop_size=768
    min_freq=80
    max_fund_freq=300
    max_freq=8000
    mfcc_size=50
    magic=43195

    features=[]
    labels=[]

    x,fs=librosa.load(file_path)
    #fs,x=wavfile.read(file_path)           

    if len(x)>=magic:
        x=x[-magic:]
    else:                                                          
        x=np.concatenate((x,[0]*(magic-len(x))))

    x=x/32768                                               
    s=librosa.feature.melspectrogram(y=x,sr=fs,n_fft=win_size,hop_length=hop_size)
    mfcc=librosa.feature.mfcc(S=librosa.power_to_db(s),sr=fs,n_mfcc=mfcc_size)
    rms=librosa.feature.rms(y=x,frame_length=win_size,hop_length=hop_size)
    zcr=librosa.feature.zero_crossing_rate(y=x,frame_length=win_size,hop_length=hop_size)
    centroid=librosa.feature.spectral_centroid(y=x,sr=fs,n_fft=win_size,hop_length=hop_size)
                
    #pitch         (write explanation)
                
    min_lag=int(fs/max_fund_freq)
    max_lag=int(fs/min_freq)
    L=range(min_lag,max_lag+1)
    spec=librosa.core.stft(x,n_fft=win_size,hop_length=hop_size,win_length=win_size)
    dividend=np.transpose([np.real(np.fft.ifft(row)) for row in (np.absolute(spec)**2).transpose()])
    divisor=np.transpose([win_size-lag+1 for lag in L])
    acf=dividend[L]/divisor[:,None]
    i_max=np.argmax(acf,axis=0)
    pitch=fs/(i_max-1+min_lag)
                
    if len(set([len(mfcc[0]),len(rms[0]),len(zcr[0]),len(centroid[0]),len(pitch)]))!=1:         
        print(' Error: File '+file+' has different numbers of windows among different features!')     

    gender=np.vstack(([1]*len(rms[0]),[0]*len(rms[0])))          

    concat=np.vstack((mfcc,rms,zcr,centroid,pitch,gender))
    features.append(concat)
    features=np.array(features)

    features=features.reshape(features.shape[0],features.shape[1],features.shape[2],1)    

    return features                                          #predict on the returned data using the saved model

In [6]:
ftrs=extract_features("record.wav")
ftrs.shape

(1, 56, 57, 1)

In [10]:
ftrs

array([[[[-1.11609973e+03],
         [-1.10321973e+03],
         [-1.10414648e+03],
         ...,
         [-1.13137085e+03],
         [-1.13137085e+03],
         [-1.13137085e+03]],

        [[ 2.06819115e+01],
         [ 3.61486053e+01],
         [ 3.52445984e+01],
         ...,
         [ 0.00000000e+00],
         [ 0.00000000e+00],
         [ 0.00000000e+00]],

        [[ 1.80772362e+01],
         [ 2.67366905e+01],
         [ 2.67803288e+01],
         ...,
         [ 0.00000000e+00],
         [ 0.00000000e+00],
         [ 0.00000000e+00]],

        ...,

        [[ 1.37812500e+02],
         [ 2.82692308e+02],
         [ 2.82692308e+02],
         ...,
         [ 1.35276074e+02],
         [ 2.10000000e+02],
         [ 2.59411765e+02]],

        [[ 1.00000000e+00],
         [ 1.00000000e+00],
         [ 1.00000000e+00],
         ...,
         [ 1.00000000e+00],
         [ 1.00000000e+00],
         [ 1.00000000e+00]],

        [[ 0.00000000e+00],
         [ 0.00000000e+00],
         [

In [11]:
np.save("sample.npy",ftrs)